## Aquire weather data for test_flights

In [1]:
import pandas as pd
import numpy as np

In [2]:
import datetime as dt

In [3]:
import requests
import time

### Create date/city unique combos table

In [5]:
flight_unique = pd.read_csv('data/flights_test_uniques.csv', index_col=0)

In [6]:
flight_unique.head()

,fl_date,city,count,weather_categ
758,2020-01-03,"Chicago, IL",2591,empty
408,2020-01-02,"Chicago, IL",2587,empty
1463,2020-01-05,"Chicago, IL",2556,empty
1813,2020-01-06,"Chicago, IL",2556,empty
2166,2020-01-07,"Chicago, IL",2413,empty


### Import lat long table
table was cleaned in EDA_Task3 notebook

In [7]:
#api requires lat/long, found a csv of american cities and lat/longs
latlong_df = pd.read_csv('data/uscities_ll.csv')

In [8]:
latlong_df.shape

(30409, 6)

In [9]:
latlong_df

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"
...,...,...,...,...,...,...
30404,Drummond,ID,43.9996,-111.3433,America/Boise,"Drummond, ID"
30405,Lost Springs,WY,42.7652,-104.9255,America/Denver,"Lost Springs, WY"
30406,Provo,SD,43.1937,-103.8329,America/Denver,"Provo, SD"
30407,Goldcreek,MT,46.5838,-112.9284,America/Denver,"Goldcreek, MT"


In [10]:
latlong_df['city_state'].nunique()

30351

In [11]:
latlong_df = latlong_df.drop_duplicates(subset='city_state', keep="first")

In [12]:
latlong_df

,city,state_id,lat,lng,timezone,city_state
0,New York,NY,40.6943,-73.9249,America/New_York,"New York, NY"
1,Los Angeles,CA,34.1141,-118.4068,America/Los_Angeles,"Los Angeles, CA"
2,Chicago,IL,41.8375,-87.6866,America/Chicago,"Chicago, IL"
3,Miami,FL,25.7840,-80.2101,America/New_York,"Miami, FL"
4,Dallas,TX,32.7935,-96.7667,America/Chicago,"Dallas, TX"
...,...,...,...,...,...,...
30404,Drummond,ID,43.9996,-111.3433,America/Boise,"Drummond, ID"
30405,Lost Springs,WY,42.7652,-104.9255,America/Denver,"Lost Springs, WY"
30406,Provo,SD,43.1937,-103.8329,America/Denver,"Provo, SD"
30407,Goldcreek,MT,46.5838,-112.9284,America/Denver,"Goldcreek, MT"


### API functions

In [13]:
#open weather api function
def open_weather_api(lat, long, date, timezone = "America%2FChicago"):
    """
    function returns individual api calls by location and date. 
    """
    #timezone = America%2FChicago #timezone format, opportunity to fine tune this - doesn't work for some reason, I think formatting
    
    url = f"https://archive-api.open-meteo.com/v1/era5?latitude={lat}&longitude={long}&start_date={date}&end_date={date}&hourly=precipitation,rain,snowfall,cloudcover&daily=precipitation_sum,rain_sum,snowfall_sum,precipitation_hours&timezone={timezone}"
    response = requests.get(url)
    return response.json()

In [14]:
test_api = open_weather_api("41.75","-87.75","2018-10-25", "America%2FNew_York")
test_api

{'latitude': 41.75,
 'longitude': -87.75,
 'generationtime_ms': 0.4220008850097656,
 'utc_offset_seconds': -14400,
 'timezone': 'America/New_York',
 'timezone_abbreviation': 'EDT',
 'elevation': 192.0,
 'hourly_units': {'time': 'iso8601',
  'precipitation': 'mm',
  'rain': 'mm',
  'snowfall': 'cm',
  'cloudcover': '%'},
 'hourly': {'time': ['2018-10-25T00:00',
   '2018-10-25T01:00',
   '2018-10-25T02:00',
   '2018-10-25T03:00',
   '2018-10-25T04:00',
   '2018-10-25T05:00',
   '2018-10-25T06:00',
   '2018-10-25T07:00',
   '2018-10-25T08:00',
   '2018-10-25T09:00',
   '2018-10-25T10:00',
   '2018-10-25T11:00',
   '2018-10-25T12:00',
   '2018-10-25T13:00',
   '2018-10-25T14:00',
   '2018-10-25T15:00',
   '2018-10-25T16:00',
   '2018-10-25T17:00',
   '2018-10-25T18:00',
   '2018-10-25T19:00',
   '2018-10-25T20:00',
   '2018-10-25T21:00',
   '2018-10-25T22:00',
   '2018-10-25T23:00'],
  'precipitation': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
  

In [16]:
def city_scrubber(city_str):
    """
    city name will be scrubbed to remove multiple names
    Dallas/Fort Something, TX -----> Dallas, TX
    """
    try:
        state = city_str.split(',')       #split city(s) and state
        cities = state[0].split('/')      #split the list of cities by /
        return cities[0] + ',' + state[1]
    except: 
        return "no state in your city string"

In [17]:
#test
city_scrubber("Dallas/Fort Something, TX")

'Dallas, TX'

In [18]:
city_scrubber("Austin, TX")

'Austin, TX'

In [19]:
city_scrubber("Austin")

'no state in your city string'

In [20]:
#function that returns the weather condition for a date/city
def weather_condition(city, date, lat_long_df):
    """
    returns the weather condition for a single date/city. 
    city format "city, ST"
    date format "2020-01-01"
    """
    #scrub city
    #print(city)
    city_scrub = city_scrubber(city)
    if city_scrub == 'no state in your city string':
        return city_scrub, city
    
    #pull the lat and long for city
    lat = None
    long = None
    try:
        lat = float(lat_long_df['lat'][lat_long_df['city_state'] == city_scrub])
        long = float(lat_long_df['lng'][lat_long_df['city_state'] == city_scrub])
    except:
        print(lat, long, city)
        pass
    
    #call the weather api
    if lat and long:         #none is False
        json_result = open_weather_api(lat, long, date)
        #print(json_result)
        try:
            total_rain = int(json_result['daily']['rain_sum'][0])                  # mm
        except: 
            total_rain = None
        try:
            total_snow = int(json_result['daily']['snowfall_sum'][0])              # cm        
        except:
            total_snow = None
        try:
            cloudcover_mean = round(sum(json_result['hourly']['cloudcover'])/len(json_result['hourly']['cloudcover']))     # %
        except:
            cloudcover_mean = None

        if total_rain is None and total_snow is None and cloudcover_mean is None:
            result = "no weather data"    
        elif total_rain == 0 and total_snow == 0 and cloudcover_mean < 40:
            result = "sunny"
        elif total_rain == 0 and total_snow == 0 and cloudcover_mean >= 40:
            result = "cloudy"
        elif total_rain > 0 and total_snow == 0:
            result = "rain"
        elif total_rain == 0 and total_snow > 0:
            result = "snow"
        elif total_rain > 0 and total_snow > 0:
            result = "snow & rain"
        else:
            result = f"Error, total rain: {total_rain}, total snow: {total_snow}, mean cloud: {cloudcover_mean}"

        return result  

In [21]:
latlong_df['lat'][latlong_df['city_state'] == "Denver, CO"]

18    39.762
Name: lat, dtype: float64

In [22]:
#test
weather_condition("Denver, CO", "2019-11-23", latlong_df)

'sunny'

In [26]:
from tqdm import tqdm

In [27]:
for row in tqdm(range(len(flight_unique))):
    flight_unique['weather_categ'][row] = weather_condition(flight_unique['city'][row], flight_unique['fl_date'][row], latlong_df)

  0%|                                                                                         | 0/2450 [00:00<?, ?it/s]C:\Users\3_hal\AppData\Local\Temp\ipykernel_22196\3046711196.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flight_unique['weather_categ'][row] = weather_condition(flight_unique['city'][row], flight_unique['fl_date'][row], latlong_df)
  0%|                                                                                 | 2/2450 [00:00<13:42,  2.98it/s]

None None Adak Island, AK


  1%|▌                                                                               | 16/2450 [00:05<13:55,  2.91it/s]

None None Ashland, WV


  1%|▊                                                                               | 25/2450 [00:08<17:04,  2.37it/s]

...

 95%|██████████████████████████████████████████████████████████████████████████▏   | 2331/2450 [13:09<00:42,  2.79it/s]

None None Newburgh/Poughkeepsie, NY


 97%|███████████████████████████████████████████████████████████████████████████▋  | 2379/2450 [13:25<00:24,  2.84it/s]

None None Saipan, TT


100%|██████████████████████████████████████████████████████████████████████████████| 2450/2450 [13:50<00:00,  2.95it/s]


In [28]:
flight_unique['weather_categ']

758     cloudy
408      sunny
1463    cloudy
1813     sunny
2166     sunny
         ...  
1383     sunny
209     cloudy
278      sunny
183      sunny
2365     sunny
Name: weather_categ, Length: 2450, dtype: object

In [29]:
flight_unique

,fl_date,city,count,weather_categ
758,2020-01-03,"Chicago, IL",2591,cloudy
408,2020-01-02,"Chicago, IL",2587,sunny
1463,2020-01-05,"Chicago, IL",2556,cloudy
1813,2020-01-06,"Chicago, IL",2556,sunny
2166,2020-01-07,"Chicago, IL",2413,sunny
...,...,...,...,...
1383,2020-01-04,"Vernal, UT",2,sunny
209,2020-01-01,"Moab, UT",2,cloudy
278,2020-01-01,"Salina, KS",1,sunny
183,2020-01-01,"Liberal, KS",1,sunny


In [30]:
flight_unique.to_csv('data/flights_test_uniques.csv', header=True, index=False)

In [31]:
flight_unique.groupby(by = 'weather_categ').count()

,fl_date,city,count
weather_categ,,,
cloudy,615,615,615
no weather data,28,28,28
rain,372,372,372
snow,136,136,136
snow & rain,29,29,29
sunny,1168,1168,1168


In [ ]:
#now match that to the flights_test df